In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [3]:
train = pd.read_csv('/Users/szkfzx/datasets/jigsaw-toxic-comment-classification-challenge/train.csv').fillna(' ')
test = pd.read_csv('/Users/szkfzx/datasets/jigsaw-toxic-comment-classification-challenge/test.csv').fillna(' ')

In [4]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [5]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

/Users/szkfzx/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [6]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [7]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [8]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

In [9]:
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

CV score for class toxic is 0.969219875449
CV score for class severe_toxic is 0.987588323305
CV score for class obscene is 0.983868516532
CV score for class threat is 0.983355076455
CV score for class insult is 0.97745382111
CV score for class identity_hate is 0.97396507953


In [10]:
print('Total CV score is {}'.format(np.mean(scores)))

Total CV score is 0.979241782064


In [11]:
submission.to_csv('submission.csv', index=False)

In [12]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.982242,0.087789,0.956482,0.012205,0.864928,0.087970
1,0000247867823ef7,0.027559,0.005406,0.017161,0.002533,0.016126,0.006817
2,00013b17ad220c46,0.026809,0.005302,0.015809,0.002481,0.013885,0.005709
3,00017563c3f7919a,0.013758,0.003821,0.010001,0.002263,0.010631,0.003243
4,00017695ad8997eb,0.060138,0.004322,0.022041,0.002388,0.021507,0.004860
